<a href="https://colab.research.google.com/github/Alecia113/NLP-Emotional-analysis/blob/main/AE1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AE1

In [1]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1lTD6bgRkmwguGAr30v-r0KBPdtnVneLb'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('testing_data.pkl')  

id = '1pCUdlZMoj99UZHtqFeza86fvVQfFmDFX'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('training_data.pkl')  

import pickle
training_data = pickle.load(open("training_data.pkl","rb"))
testing_data = pickle.load(open("testing_data.pkl","rb"))

print("------------------------------------")
print("Size of training dataset: {0}".format(len(training_data))) #8000
print("Size of testing dataset: {0}".format(len(testing_data))) #2000
print("------------------------------------")

print("------------------------------------")
print("Sample Data")
print("LABEL: {0} / SENTENCE: {1}".format(training_data[0][0], training_data[0][1]))
print("------------------------------------")
#LABEL: neg / SENTENCE: hopeless for tmr :(

------------------------------------
Size of training dataset: 8000
Size of testing dataset: 2000
------------------------------------
------------------------------------
Sample Data
LABEL: neg / SENTENCE: hopeless for tmr :(
------------------------------------


In [ ]:
import torch
#You can enable GPU here (cuda); or just CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 1.2. Preprocess data



*You are required to describe which data preprocessing techniques were conducted with justification of your decision. *
请您说明采用了哪些数据预处理技术，并说明决定的理由。*

In [ ]:

#预处理： 2分：
'''
数据集： NLTK's Twitter_Sample dataset 【via APIs】可以查看细节

训练和测试都被提供了(testing_data.pkl, training_data.pkl) 可以用A1 template ipynb 
，可以使用作业1模板(template)ipynb中提供的代码从Google Drive中下载。

[要求]
  预处理训练集用lab5的几个技术的合集
  (e.g. tokenisation, removing numbers, converting to lowercase, removing stop words, stemming, etc.).
  要证明为什么要用。#您应该说明您应用特定预处理技术的原因（说明您的决定）。
  【看起来有些是不能用的要测试】
  po',
  'ff',
  'follow',
  'let',
  'tri',
  'keepitloc',
  'po',
  'teen_emma',
  'br1stler',
  'jeweleyegoddess',
  'caratoyn',
  'caraa
'''

In [ ]:
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
import pprint
from nltk.stem.porter import *
#lemmatisation
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

sww = sw.words()
sww.extend(["'", '"'])

#tokenized_docs=[]
stemmer = PorterStemmer()

lemmatizer = WordNetLemmatizer()

clean_doc = re.sub(r'[^\w\s]','', str(training_data))   #removing numbers
tokenized_sentence = sent_tokenize(clean_doc.lower())   #这里明天lab5再看看 converting to lowercase
lower_case = word_tokenize(clean_doc.lower())   #tokenisation
stopword_removal = [w for w in lower_case if not w in sww]    #removing stop words
#singles = [stemmer.stem(plural) for plural in plurals]#stemming
singles = [stemmer.stem(plural) for plural in stopword_removal] #无复数
lemma_sentence = [lemmatizer.lemmatize(w) for w in singles ]
#tokenized_docs.append(lemma_sentence)
pprint.pprint(lemma_sentence)

#pprint.pprint(tokenized_docs) #58613
#所以到底要不要加这个列表，加它有啥子用


In [8]:
len(lemma_sentence)

58613

In [10]:
# Please comment your code 请评述你的代码
def preprocess(corpus):
  sww = sw.words()
  sww.extend(["'", '"'])

  #tokenized_docs=[]
  stemmer = PorterStemmer()

  lemmatizer = WordNetLemmatizer()

  clean_doc = re.sub(r'[^\w\s]','', str(corpus))   #removing numbers
  tokenized_sentence = sent_tokenize(clean_doc.lower())   #这里明天lab5再看看 converting to lowercase
  lower_case = word_tokenize(clean_doc.lower())   #tokenisation
  stopword_removal = [w for w in lower_case if not w in sww]    #removing stop words
  #singles = [stemmer.stem(plural) for plural in plurals]#stemming
  singles = [stemmer.stem(plural) for plural in stopword_removal] #无复数
  lemma_sentence = [lemmatizer.lemmatize(w) for w in singles ]
  #tokenized_docs.append(lemma_sentence)
  #pprint.pprint(lemma_sentence)
  return lemma_sentence #tokenized_docs

# 2 - Model Implementation

## 2.1. Word Embeddings

In [ ]:
#模块实现 model implementation 7分
'''
【要做的】
  1，word embedding module, 2，Lexicon Embedding module + 3，Bi-directional RNN sequence model

  [从lab2,4,5随便选参数]embeddings_dimension ; learning_rate epochs;
  [lab5 有模型构建]


# word embedding  2分 [lab2]
  创建word embedding model(表现word vectors eg: word2vec-CBOW, word2vec-Skip gram, fastText + Glove[lab2])
  input embedding of your sequence model
  [lab3,4]使用了one-hot vectors 作为inputs 给长序列模型
  【要求做的】
    1.数据的预处理（word embeddings[lab2;还要做解释说明]） NLTK Twitter dataset ( section 1 提供了) or/and 任何数据集（TED talk , Google News)
    2.创建训练模型（for word embeddings) 创建训练模型；需要说清楚超参数【lab2（dimension of embeddings; window size, learning rate,etc.】 
        note 任何word embeddings model[lab2] (word2vec-CBOW, word2vec-Skip gram, fasttext, glove)【证明】
    3.训练模型： train model

#Lexicon embedding 2分 （词典嵌入）
  每个单词的积极或者消极的lexicon【从文件夹下载：opinion Lexicaon】(2006 positive and 4783 negative words)
  每个单词要被改成： one-dimensional一维分类嵌入，三类。【eg:[not_exist(0), negative(1), positive(2);这个012类别 会在[2.3 的Bi-directional RNN 模型中用到]]
  【如果你想用超过一维，或者不使用分类嵌入（categorical） 证明观点】

# Bi-directional RNN Sequence Model 3分
Many-to-One (N to 1) 建个n to 1 sequence model 为了探测感情（detect sentiment/emotion）你的模型应该是评价中选出的最佳模型（将在第3节评价【evaluation】中讨论）。
  【做的】
    1.apply/import word + lexicon embedding as input: 你要把训练好的词嵌入和词库嵌入合并起来，并应用到序列模型上    ##不太理解
    2.1构建训练Sequence model: 构建Bi-directional RNN-based(Bi-RNN / Bi-LSTM / Bi-GRU) 
    2.2Many to One (N to 1) sequence model (N: word, One: sentiment- positive or negative)
    3.描述超参【证明】【lab4,5】[the number of epochs, learning rate ……]
    4.Train model  展示：training loss + number of epochs[lab4,lab5]

    【不展示没分】
'''
'''
【要做的】
  1，word embedding module, 2，Lexicon Embedding module + 3，Bi-directional RNN sequence model

  [从lab2,4,5随便选参数]embeddings_dimension ; learning_rate epochs;
  [lab5 有模型构建]


# word embedding  2分 [lab2]
  创建word embedding model(表现word vectors eg: word2vec-CBOW, word2vec-Skip gram, fastText + Glove[lab2])
  input embedding of your sequence model
  [lab3,4]使用了one-hot vectors 作为inputs 给长序列模型
  【要求做的】
    1.数据的预处理（word embeddings[lab2;还要做解释说明]） NLTK Twitter dataset ( section 1 提供了) or/and 任何数据集（TED talk , Google News)
    2.创建训练模型（for word embeddings) 创建训练模型；需要说清楚超参数【lab2（dimension of embeddings; window size, learning rate,etc.】 
        note 任何word embeddings model[lab2] (word2vec-CBOW, word2vec-Skip gram, fasttext, glove)【证明】
    3.训练模型： train model
    '''

*You are required to describe which model was implemented (i.e. Word2Vec with CBOW, FastText with SkipGram, etc.) with justification of your decision *
*要求您说明采用的是哪种模式（即Word2Vec与CBOW、FastText与SkipGram等），并说明您决定的理由*。【也就是三种都要做一做】

In [ ]:
# Please comment your code

### 2.1.1. Data Preprocessing for Word Embeddings 字词嵌入的数据预处理

*You are required to describe which preprocessing techniques were used with justification of your decision.*
您需要说明使用了哪些预处理技术，并说明您的决定的理由。

In [ ]:
# Please comment your code

### 2.1.2. Build Word Embeddings Model. 建立单词嵌入模型

*You are required to describe how hyperparameters were decided with justification of your decision.*您需要说明超参数是如何决定的，并说明决定的理由。

In [ ]:
# Please comment your code

### 2.1.3. Train Word Embeddings Model

In [ ]:
# Please comment your code

#Test


In [10]:
testing_data[0]#('neg', '@AndreaMarySmith very helpful .... Or will be once I stop crying :(')
'''
都是标签+句子
'''

('neg', '@AndreaMarySmith very helpful .... Or will be once I stop crying :(')

In [9]:
training_data[0]#('neg', 'hopeless for tmr :(')
training_data[1]#('neg',"Everything in the kids section of IKEA is so cute. Shame I'm nearly 19 in 2 months :(")
training_data[2]#('neg', '@Hegelbon That heart sliding into the waste basket. :(')

('neg', '@Hegelbon That heart sliding into the waste basket. :(')

In [6]:
len(testing_data)

2000

In [ ]:
testing_data  #2000
'''
 "@NUFC_Vine he's gona miss Sheff Utd/York probably waiting for a visa which leaves gim with 1 friendly. Bet he starts season on the bench :("),
 ('neg', "@ranaPTX_ BUT WHO WILL LEAD US WHEN YOU'RE GONE :(("),
 ('neg', '@LottyStorer @charliee_catlin 😂😩😩 what a mess :('),
 ('neg', '@twinitisha irony is its more harmful .. :('),
 ('neg', '@DollyyDaydreams I miss you so much more :( xxx'),
 ('neg', 'no motivation :('),
 ('neg',
  '@LiLGeekette but that is the point. Because it is YOLO I sort of die when lactose is in me... Ugh like 3 months pregnant bloating :('),
 (
'''

In [4]:
len(training_data)

8000

In [3]:
training_data #8000
'''
 ('neg', "@rupexo @LordOfTheMics fuckkk it's a Wednesday :("),
 ('neg',
  '@PetiteMistress DO IT! I want to start one for making small games, but I feel like I need to get a jump start before asking for support :('),
 ('neg', 'Justin where are you ? :( @justinbieber'),
 ('neg', ":( :( I'm gonna cry"),
 ('neg',
  "@IanHallard @ArtsTheatreLDN I'm really excited but also really sad because once again I won't be able to see it :("),
 ('neg', "I can't finish my Sanum today! :(\n#LLAOLLAO #Dessert but too full"),
 ('neg', '@jxstkatie @TheSvante i want foood :(('),
 ('neg', 'Last day :('),
 ('neg', '#UberIceCream was super! But we didnt get the glares :( @Uber_Pune'),
 ('neg', "@ffyeahh i don't know how to do vines :("),
 ('neg',
  "@selenagomez you was Tweetin earlier this morning &amp; i wasn't here :( sad mood all the day :("),
 ('n
 '''

'\n (\'neg\', "@rupexo @LordOfTheMics fuckkk it\'s a Wednesday :("),\n (\'neg\',\n  \'@PetiteMistress DO IT! I want to start one for making small games, but I feel like I need to get a jump start before asking for support :(\'),\n (\'neg\', \'Justin where are you ? :( @justinbieber\'),\n (\'neg\', ":( :( I\'m gonna cry"),\n (\'neg\',\n  "@IanHallard @ArtsTheatreLDN I\'m really excited but also really sad because once again I won\'t be able to see it :("),\n (\'neg\', "I can\'t finish my Sanum today! :(\n#LLAOLLAO #Dessert but too full"),\n (\'neg\', \'@jxstkatie @TheSvante i want foood :((\'),\n (\'neg\', \'Last day :(\'),\n (\'neg\', \'#UberIceCream was super! But we didnt get the glares :( @Uber_Pune\'),\n (\'neg\', "@ffyeahh i don\'t know how to do vines :("),\n (\'neg\',\n  "@selenagomez you was Tweetin earlier this morning &amp; i wasn\'t here :( sad mood all the day :("),\n (\'n\n '